# Free form questioning on COVID-19 dataset 

>We will use universal sentence encoder to encode text from COVID-19 dataset and use to answer queries

- Based on the paper at: https://arxiv.org/abs/1803.11175

- Dataset available at: https://pages.semanticscholar.org/coronavirus-research

- By Dattaraj J Rao (Persistent Systems) - https://www.linkedin.com/in/dattarajrao

## We will use the Sentence Transformers library

Approach is to encode relevant text corpus from COVID-19 dataset and then match the question embedding with this to fiond top 3 matching answers.

In [1]:
#hide
pip install -U sentence-transformers

     |████████████████████████████████| 61kB 2.3MB/s eta 0:00:011
     |████████████████████████████████| 450kB 3.1MB/s eta 0:00:01
     |████████████████████████████████| 81.1MB 54.3MB/s eta 0:00:01     |███████████                     | 27.6MB 38.1MB/s eta 0:00:02
     |████████████████████████████████| 15.1MB 15.8MB/s eta 0:00:01
     |████████████████████████████████| 28.4MB 17.0MB/s eta 0:00:01
     |████████████████████████████████| 1.5MB 66.9MB/s eta 0:00:01
     |████████████████████████████████| 686kB 35.7MB/s eta 0:00:01
     |████████████████████████████████| 870kB 64.4MB/s eta 0:00:01
     |████████████████████████████████| 1.1MB 54.3MB/s eta 0:00:01
     |████████████████████████████████| 133kB 69.1MB/s eta 0:00:01
     |████████████████████████████████| 71kB 32.4MB/s eta 0:00:01
     |████████████████████████████████| 6.0MB 60.2MB/s eta 0:00:01
  Created wheel for sentence-transformers: filename=sentence_transformers-0.2.5.1-cp37-none-any.whl size=67079 sha256=395f340545f

### Read the dataset, create embeddings vector

In [2]:
#hide
import os
import json
import warnings
warnings.simplefilter('ignore')

JSON_PATH = 'CORD-19-research-challenge/2020-03-13/noncomm_use_subset/noncomm_use_subset/'

json_files = [pos_json for pos_json in os.listdir(JSON_PATH) if pos_json.endswith('.json')]

corpus = []

# loop through the files
for jfile in json_files[::]:
    # for each file open it and read as json
    with open(os.path.join(JSON_PATH, jfile)) as json_file:
        covid_json = json.load(json_file)
        # read abstract
        for item in covid_json['abstract']:
            corpus.append(item['text'])
        # read body text
        for item in covid_json['body_text']:
            corpus.append(item['text'])
            
print("Corpus size = %d"%(len(corpus)))

from sentence_transformers import SentenceTransformer
import scipy

embedder = SentenceTransformer('bert-base-nli-mean-tokens')
corpus_embeddings = embedder.encode(corpus)

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/CORD-19-research-challenge/2020-03-13/noncomm_use_subset/noncomm_use_subset/'

In [ ]:
#hide
def ask_question(query):
    queries = [query]
    query_embeddings = embedder.encode(queries)

    # Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
    closest_n = 5
    for query, query_embedding in zip(queries, query_embeddings):
        distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings, "cosine")[0]

        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: x[1])
        
        # get the closest answers
        for idx, distance in results[0:closest_n]:
            print("\n\n======================\n\n")        
            print("ANSWER = \n", corpus[idx].strip(), "(Score: %.4f)" % (1-distance))

## Question 1: Does smoking or pre-existing pulmonary disease increase risk of COVID-19?

In [ ]:
ask_question('Does smoking or pre-existing pulmonary disease increase risk of COVID-19?')

## Question 2: Are neonates and pregnant women ar greater risk of COVID-19?

In [ ]:
ask_question('Are neonates and pregnant women ar greater risk of COVID-19?')

## Question3: Severity of disease, including risk of fatality among symptomatic hospitalized patients, and high-risk patient groups

In [ ]:
ask_question('Severity of disease, including risk of fatality among symptomatic hospitalized patients, and high-risk patient groups')

## Question 4: Socio-economic and behavioral factors to understand the economic impact of the virus and whether there were differences.

In [ ]:
ask_question('Socio-economic and behavioral factors to understand the economic impact of the virus and whether there were differences.')

## Question 5: Severity of disease, including risk of fatality among symptomatic hospitalized patients, and high-risk patient groups

In [ ]:
ask_question('Severity of disease, including risk of fatality among symptomatic hospitalized patients, and high-risk patient groups')

# Ask your own question

In [3]:
ask_question(input())

NameError: name 'ask_question' is not defined